In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.9 MB/s eta 0:00:00


In [1]:
!pip install sentence_transformers

In [3]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from scipy.stats import uniform
from sentence_transformers import SentenceTransformer

# Mount Google Drive
drive.mount('/content/drive')

# Load the SBERT model
sbert_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Function to convert text to SBERT embeddings
def convert_to_sbert_embeddings(input_file, text_column):
    # Read the Excel file
    df = pd.read_excel(input_file)

    # Function to get SBERT embeddings for a text
    def get_sbert_embeddings(text):
        return sbert_model.encode(text, convert_to_numpy=True)

    # Generate embeddings for the text column
    embeddings = df[text_column].apply(lambda x: get_sbert_embeddings(str(x)))

    # Stack embeddings into a 2D array
    embeddings = np.stack(embeddings.values)

    # Print the dimensions of the embeddings
    print(f"SBERT embeddings dimensions for '{text_column}': {embeddings.shape}")

    # Return the embeddings and the original DataFrame
    return embeddings, df

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Define individual model functions
def knn_model(X_train, y_train, X_test, y_test):
    model = KNeighborsClassifier()
    model.fit(X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    model = MLPClassifier(max_iter=300)
    model.fit(X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    model = XGBClassifier(eval_metric='mlogloss')
    model.fit(X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    model = AdaBoostClassifier(algorithm='SAMME')
    model.fit(X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    model = CatBoostClassifier(verbose=0)
    model.fit(X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    model = LogisticRegression(max_iter=300)
    model.fit(X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    model = SVC()
    model.fit(X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Define RandomizedSearchCV for hyperparameter tuning for RandomForest
def random_forest_tune(X_train, y_train):
    param_dist = {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, 30, None],
        'min_samples_split': uniform(0.01, 0.1),
        'min_samples_leaf': uniform(0.01, 0.1)
    }
    rf = RandomForestClassifier()
    random_search = RandomizedSearchCV(rf, param_dist, n_iter=30, cv=5, verbose=1, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best RandomForest Parameters: {random_search.best_params_}")
    return random_search.best_estimator_

# Define a master function to call all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }

    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input and output file paths
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'

# Specify the text column for SBERT embeddings
text_column = 'cleaned_answer'

# Convert text column to SBERT embeddings and get the original DataFrame
embeddings, original_data = convert_to_sbert_embeddings(input_file, text_column)

# Modify y to make sure the labels start from 0 (shift -1 to 0)
y = original_data['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(embeddings, y, test_size=0.3, random_state=42)

# Run hyperparameter tuning for Random Forest with RandomizedSearchCV
best_rf_model = random_forest_tune(X_train, y_train)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
SBERT embeddings dimensions for 'cleaned_answer': (681, 384)
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best RandomForest Parameters: {'max_depth': 30, 'min_samples_leaf': 0.049280944708952806, 'min_samples_split': 0.04065747972683085, 'n_estimators': 100}
Running KNN model...
KNN Model Metrics:
Train Accuracy: 0.7122
Train Precision: 0.6932
Train Recall: 0.7122
Train F1 Score: 0.6892
Train Confusion Matrix:
[[  4  11  17]
 [  1  62  69]
 [  2  37 273]]

Test Accuracy: 0.6878
Test Precision: 0.6685
Test Recall: 0.6878
Test F1 Score: 0.6740
Test Confusion Matrix:
[[  2   2   9]
 [  2  13  26]
 [  1  24 126]]

Running Random Forest model...
Random Forest Model Metrics:
Train Accuracy: 0.9769
Train Precision: 0.9771
Train Recall: 0.9769
Train F1 Score: 0.9768
Train Confusion Matrix:
[[ 31   0   1]
 [  0 124   8]
 [  0   2 310]]

Test Accuracy: